#Create shared functions

Please copy all the notebooks of this project to your Google Drive into a folder named "notebooks" inside a folder of your choice. Please also set the "notebook_path" variable in every notebook to this folder. This script will generate the folder structure for the data generated by the project.

- base folder
    - notebooks
        - 0_define_helper_functions.ipynb = THIS NOTEBOOK
        - 1a_generate_datasets.ipynb
        - ...
    - data (will be created)
    - quantumflow (will be created / overwritten)


In [ ]:
notebook_path = "Projects/QuantumFlow/notebooks"
import os

try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir("/content/gdrive/My Drive/" + notebook_path)
except:
    pass

if not os.path.exists('../data'):
    os.makedirs('../data')

if not os.path.exists('../quantumflow'):
    os.makedirs('../quantumflow')

In [ ]:
%%writefile ../quantumflow/generate_datasets.py
import tensorflow as tf
import os
import pickle
from quantumflow.colab_utils import load_hyperparameters, integrate, laplace
from quantumflow.numerov_solver import solve_schroedinger

@tf.function
def generate_potentials(return_x=False,
                        return_h=False,
                        dataset_size=100, 
                        discretisation_points=500, 
                        n_gauss=3, 
                        interval_length=1.0,
                        a_minmax=(0.0, 3*10.0), 
                        b_minmax=(0.4, 0.6), 
                        c_minmax=(0.03, 0.1), 
                        n_method='sum',
                        dtype='float64',
                        **kwargs):
    
    if dtype == 'double' or dtype == 'float64':
        dtype = tf.float64
    elif dtype == 'float':
        dtype = tf.float32
    else:
        raise ValueError('unknown dtype {}'.format(dtype))
    

    x = tf.linspace(tf.constant(0.0, dtype=dtype), interval_length, discretisation_points, name="x")

    a = tf.random.uniform((dataset_size, 1, n_gauss), minval=a_minmax[0], maxval=a_minmax[1], dtype=dtype, name="a")
    b = tf.random.uniform((dataset_size, 1, n_gauss), minval=b_minmax[0]*interval_length, maxval=b_minmax[1]*interval_length, dtype=dtype, name="b")
    c = tf.random.uniform((dataset_size, 1, n_gauss), minval=c_minmax[0]*interval_length, maxval=c_minmax[1]*interval_length, dtype=dtype, name="c")

    curves = -tf.square(tf.expand_dims(tf.expand_dims(x, 0), 2) - b)/(2*tf.square(c))
    curves = -a*tf.exp(curves)

    if n_method == 'sum':
        potentials = tf.reduce_sum(curves, -1, name="potentials")
    elif n_method == 'mean':
        potentials = tf.reduce_mean(curves, -1, name="potentials")
    else:
        raise NotImplementedError('Method {} is not implemented.'.format(n_method))

    returns = [potentials]

    if return_x:
        returns += [x]
    
    if return_h:
        h = tf.cast(interval_length/(discretisation_points-1), dtype=dtype) # discretisation interval
        returns += [h]
   
    return returns

def generate_datasets(data_dir, experiment, generate_names):
    if not isinstance(generate_names, list):
        generate_names = [generate_names]

    base_dir = os.path.join(data_dir, experiment)
    file_hyperparams = os.path.join(base_dir, "hyperparams.config")

    for run_name in generate_names:
        params = load_hyperparameters(file_hyperparams, run_name=run_name, globals=globals())

        tf.set_random_seed(params['seed'])
        potential, x, h = generate_potentials(return_x=True, return_h=True, **params)

        params['h'] = h
        energies, wavefunctions = solve_schroedinger(potential, params)
        
        with open(os.path.join(base_dir, params['filename'] + '.pkl'), 'wb') as f:
            pickle.dump({'x': x.numpy(), 'h': h.numpy(), 'potential': potential.numpy(), 'wavefunctions': wavefunctions.numpy(), 'energies': energies.numpy()}, f)

        print("dataset", params['filename'] + '.pkl', "saved to", base_dir)


In [ ]:
%%writefile ../quantumflow/numerov_solver.py
import tensorflow as tf
from quantumflow.colab_utils import integrate

# recurrent tensorflow cell for solving the numerov equation recursively
class ShootingNumerovCell(tf.keras.layers.AbstractRNNCell):
    def __init__(self, shape, h, **kwargs):
        super(ShootingNumerovCell, self).__init__(**kwargs)
        self._h2_scaled = 1 / 12 * h ** 2
        self.shape = shape

    @property
    def state_size(self):
        return self.shape + (4,)

    @property
    def output_size(self):
        return self.shape + (1,)

    def build(self, input_shape):
        self.built = True

    def call(self, inputs, states):
        k_m2, k_m1, y_m2, y_m1 = tf.unstack(states[0], axis=-1)
        
        y = (2 * (1 - 5 * self._h2_scaled * k_m1) * y_m1 - (1 + self._h2_scaled * k_m2) * y_m2) / (1 + self._h2_scaled * inputs)

        new_state = tf.stack([k_m1, inputs, y_m1, y], axis=-1)
        return y, new_state


# tf function for using the shooting numerov method
#
# the numerov_init_slope is the slope of the solution at x=0
# it can be constant>0 because it's actual value will be determined when the wavefunction is normalized
#
def shooting_numerov(k_squared, params):
    h = params['h']
    numerov_init_slope = params['numerov_init_slope']
    init_values = tf.zeros_like(k_squared[:, 0])
    one_step_values = numerov_init_slope * h * tf.ones_like(k_squared[:, 0])
    init_state = tf.stack([k_squared[:, 0], k_squared[:, 1], init_values, one_step_values], axis=-1)
    outputs = tf.keras.layers.RNN(ShootingNumerovCell(k_squared.shape[2:], h), return_sequences=True)(k_squared[:, 2:], initial_state=init_state)
    output = tf.concat([tf.expand_dims(init_values, axis=1), tf.expand_dims(one_step_values, axis=1), outputs], axis=1)
    return output


# returns the rearranged schroedinger equation term in the numerov equation
# k_squared = 2*m_e/h_bar**2*(E - V(x))
def numerov_k_squared(potentials, energies):
    return 2 * (tf.expand_dims(energies, axis=1) - tf.repeat(tf.expand_dims(potentials, axis=2), energies.shape[1], axis=2))


@tf.function
def find_split_energies(potentials, params):
    M = potentials.shape[0]
    N = params['n_orbitals']

    # Knotensatz: number of roots = quantum state
    # so target root = target excited state quantum number
    target_roots = tf.repeat(tf.expand_dims(tf.range(N + 1), axis=0), M, axis=0)

    # lowest value of potential as lower bound
    E_split = tf.repeat(tf.expand_dims(tf.reduce_min(potentials, axis=1), axis=1), N + 1, axis=1)

    solutions_split = tf.zeros((potentials.shape[0], potentials.shape[1], N + 1), dtype=potentials.dtype)
    not_converged = tf.ones(potentials.shape[0], dtype=tf.bool)
    search_boost = tf.ones_like(E_split, dtype=tf.bool)
    E_delta = tf.ones_like(E_split)

    while tf.math.reduce_any(not_converged):
        V_split = numerov_k_squared(tf.boolean_mask(potentials, not_converged), tf.boolean_mask(E_split, not_converged))

        solutions_split_new = shooting_numerov(V_split, params)

        partitioned_data = tf.dynamic_partition(solutions_split, tf.cast(not_converged, tf.int32) , 2)
        condition_indices = tf.dynamic_partition(tf.range(tf.shape(solutions_split)[0]), tf.cast(not_converged, tf.int32) , 2)

        solutions_split = tf.dynamic_stitch(condition_indices, [partitioned_data[0], solutions_split_new])
        solutions_split.set_shape((potentials.shape[0], potentials.shape[1], N + 1))

        roots_split = tf.reduce_sum(tf.cast(detect_roots(solutions_split), tf.int32), axis=1)

        not_converged = tf.logical_and(tf.logical_not(tf.reduce_all(tf.equal(roots_split, target_roots), axis=1)), not_converged)

        search_direction = tf.cast(roots_split < target_roots, potentials.dtype) - tf.cast(roots_split > target_roots, potentials.dtype)
        boost = tf.logical_and(tf.equal(search_direction, tf.sign(E_delta)), search_boost)

        E_delta += tf.cast(boost, potentials.dtype)*E_delta
        stop_boost = search_direction * tf.sign(E_delta) < 0
        search_boost &= tf.logical_not(stop_boost)
        E_delta += -1.5*E_delta*tf.cast(stop_boost, potentials.dtype)

        E_split += E_delta*tf.expand_dims(tf.cast(not_converged, potentials.dtype), axis=-1)

    return E_split


def detect_roots(array):
    return tf.logical_or(tf.equal(array[:, 1:], 0), array[:, 1:] * array[:, :-1] < 0)


@tf.function
def solve_numerov(potentials, target_roots, split_energies, params):
    E_low = split_energies[:, :-1]
    E_high = split_energies[:, 1:]

    # because the search interval is halved at every step
    # 32 iterations will always converge to the best numerically possible accuracy of E
    # (empirically ~25 steps)

    E = 0.5 * (E_low + E_high)
    E_last = E * 2
    
    while tf.reduce_any(tf.logical_not(tf.equal(E_last, E))):
        V = numerov_k_squared(potentials, E)

        solutions = shooting_numerov(V, params)
        roots = tf.reduce_sum(tf.cast(detect_roots(solutions), tf.int32), axis=1)

        update_low = roots <= target_roots
        update_high = tf.logical_not(update_low)

        E_low = tf.where(update_low, E, E_low)
        E_high = tf.where(update_high, E, E_high)

        E_last = E
        E = 0.5 * (E_low + E_high)

    solutions_low = shooting_numerov(numerov_k_squared(potentials, E_low), params)
    roots_low = tf.cast(detect_roots(solutions_low), tf.double)

    solutions_high = shooting_numerov(numerov_k_squared(potentials, E_high), params)
    roots_high = tf.cast(detect_roots(solutions_high), tf.double)

    roots_diff = tf.abs(roots_high - roots_low)  

    roots_cumsum = tf.cumsum(tf.pad(roots_diff, ((0, 0), (1, 0), (0, 0)), 'constant'), axis=1)

    invalid = tf.equal(roots_cumsum, tf.expand_dims(roots_cumsum[:, -1], axis=1))

    return solutions_low, E, invalid


@tf.function
def solve_schroedinger(potentials, params):
    M = potentials.shape[0]
    G = potentials.shape[1]
    N = params['n_orbitals']
    
    E_split = find_split_energies(potentials, params)

    target_roots = tf.repeat(tf.expand_dims(tf.range(N), axis=0), M, axis=0)
    solutions_forward, E_forward, invalid_forward = solve_numerov(potentials, target_roots, E_split, params)
    #solutions_forward /= tf.expand_dims(tf.reduce_max(tf.abs(solutions_forward)*tf.cast(tf.logical_not(invalid_forward), tf.double), axis=1), axis=1)

    solutions_backward, E_backward, invalid_backward = solve_numerov(tf.reverse(potentials, axis=[1]), target_roots, E_split, params)
    solutions_backward = tf.reverse(solutions_backward, axis=[1])
    invalid_backward = tf.reverse(invalid_backward, axis=[1])
    #solutions_backward /= tf.expand_dims(tf.reduce_max(tf.abs(solutions_backward)*tf.cast(tf.logical_not(invalid_backward), tf.double), axis=1), axis=1)

    n_invalid_forward = tf.reduce_sum(tf.cast(invalid_forward, tf.int32), axis=1)
    n_invalid_backward = tf.reduce_sum(tf.cast(invalid_backward, tf.int32), axis=1)
    merge_index = (G - n_invalid_forward - n_invalid_backward)//2 + n_invalid_forward

    merge_value_forward = tf.reduce_sum(tf.gather(tf.transpose(solutions_forward, perm=[0, 2, 1]), tf.expand_dims(merge_index, axis=2), batch_dims=2), axis=2)
    merge_value_backward = tf.reduce_sum(tf.gather(tf.transpose(solutions_backward, perm=[0, 2, 1]), tf.expand_dims(merge_index, axis=2), batch_dims=2), axis=2)

    factor = merge_value_forward/merge_value_backward
    solutions_backward *= tf.expand_dims(factor, axis=1)

    join_mask = tf.expand_dims(tf.expand_dims(tf.range(G), axis=0), axis=2) < tf.expand_dims(merge_index, axis=1)

    solutions = tf.where(join_mask, solutions_forward, solutions_backward)

    #normalization
    density = solutions ** 2
    norm = integrate(density, params['h'])
    solutions *= 1 / tf.sqrt(tf.expand_dims(norm, axis=1))

    E = 0.5*(E_forward + E_backward)
    
    return E, solutions

In [ ]:
%%writefile ../quantumflow/colab_utils.py
import numpy as np

def integrate(y, h):
    return h*tf.reduce_sum((y[:, :-1] + y[:, 1:])/2., axis=1, name='trapezoidal_integral_approx')

def laplace(data, h):  # time_axis=1
    temp_laplace = 1 / h ** 2 * (data[:, :-2, :] + data[:, 2:, :] - 2 * data[:, 1:-1, :])
    return tf.pad(temp_laplace, ((0, 0), (1, 1), (0, 0)), 'constant')


def test_colab_devices():
    import os
    import tensorflow as tf

    has_gpu = False
    has_tpu = False

    has_gpu = (tf.test.gpu_device_name() == '/device:GPU:0')

    try:
        device_name = os.environ['COLAB_TPU_ADDR']
        has_tpu = True
    except KeyError:
        pass

    return has_gpu, has_tpu

def get_resolver():
    import os
    import tensorflow as tf

    try:
        device_name = os.environ['COLAB_TPU_ADDR']
        TPU_WORKER = 'grpc://' + device_name
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TPU_WORKER)
        tf.config.experimental_connect_to_host(resolver.master())
        tf.tpu.experimental.initialize_tpu_system(resolver)

    except KeyError:
        resolver = None

    return resolver

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)/ float(N))
    return cumsum[N:] - cumsum[:-N]

def load_hyperparameters(file_hyperparams, run_name='default', globals=None):
    from ruamel.yaml import YAML

    if globals is not None:
        with open(file_hyperparams) as f:
            globals_list = YAML().load(f)['globals']

    with open(file_hyperparams) as f:
        hparams = YAML().load(f)[run_name]

    if globals is None:
        return hparams

    dicts = [hparams]
    while len(dicts) > 0:
        data = dicts[0]
        for idx, obj in enumerate(data):
            if isinstance(data[obj], dict):
                dicts.append(data[obj])
                continue

            if data[obj] in globals_list:
                data[obj] = globals[data[obj]]
        del dicts[0]
    return hparams


import ipywidgets as widgets
from IPython.display import Audio, HTML, display
from matplotlib import animation, rc
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'

def anim_plot(array, x=None, interval=100, bar="", figsize=(15, 3), **kwargs):
    frames = len(array)
    
    if not bar == "":
        import ipywidgets as widgets
        widget = widgets.IntProgress(min=0, max=frames, description=bar, bar_style='success',
                                     layout=widgets.Layout(width='92%'))
        display(widget)

    fig, ax = plt.subplots(figsize=figsize)
    
    if x is None:
        plt_h = ax.plot(array[0], **kwargs)
    else:
        plt_h = ax.plot(x, array[0], **kwargs) 
        
    min_last = np.min(array[-1])
    max_last = np.max(array[-1])
    span_last = max_last - min_last
        
    ax.set_ylim([min_last - span_last*0.2, max_last + span_last*0.2])

    def init():
        return plt_h

    def animate(f):
        if not bar == "":
            widget.value = f

        for i, h in enumerate(plt_h):
            if x is None:
                h.set_data(np.arange(len(array[f][:, i])), array[f][:, i], **kwargs)
            else:
                h.set_data(x, array[f][:, i], **kwargs)
        return plt_h

    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=frames, interval=interval,
                                   blit=True, repeat=False)

    plt.close(fig)
    rc('animation', html='html5')
    display(HTML(anim.to_html5_video(embed_limit=1024)))

    if not bar == "":
        widget.close()

class QFDataset():
    def __init__(self, dataset_file, params, set_h=False, set_shapes=False, set_mean=False):
        import pickle
        import numpy as np

        with open(dataset_file, 'rb') as f:
            x, h, potential, wavefunctions, energies = pickle.load(f).values()
            self.dataset_size, self.discretisation_points, self.max_N = wavefunctions.shape
            assert(params['N'] <= self.max_N)
            density = np.sum(np.square(wavefunctions)[:, :, :params['N']], axis=-1)

            potential_energy_densities = np.expand_dims(potential, axis=2)*wavefunctions**2
            potential_energies = h * (np.sum(potential_energy_densities, axis=1) - 0.5 * (np.take(potential_energy_densities, 0, axis=1) + np.take(potential_energy_densities, -1, axis=1)))
            kinetic_energies = energies - potential_energies

            energy = np.sum(energies[:, :params['N']], axis=-1)
            kinetic_energy = np.sum(kinetic_energies[:, :params['N']], axis=-1)


        if params['dtype'] == 'double' or params['dtype'] == 'float64':
            if potential.dtype == np.float32:
                raise ImportError("requested dtype={}, but dataset is saved with dtype={}, which is less precise.".format(params['dtype'], potential.dtype))
            self.dtype = np.float64
        elif params['dtype'] == 'float' or params['dtype'] == 'float32':
            self.dtype = np.float32
        else:
            raise ValueError('unknown dtype {}'.format(params['dtype']))

        self.x = x.astype(self.dtype)
        self.h = h.astype(self.dtype)
        self.potential = potential.astype(self.dtype)
        self.density = density.astype(self.dtype)
        self.energy = energy.astype(self.dtype)
        self.kinetic_energy = kinetic_energy.astype(self.dtype)
        self.derivative = -self.potential

        if not 'features' in params or not 'targets' in params: 
            return

        self.features = {}
        self.targets = {}

        def add_by_name(dictionary, name):
            if name == 'density':
                dictionary['density'] = self.density
            elif name == 'derivative':
                dictionary['derivative'] = self.derivative
            elif name == 'potential':
                dictionary['potential'] = self.potential
            elif name == 'kinetic_energy':
                dictionary['kinetic_energy'] = self.kinetic_energy
            else:
                raise KeyError('feature/target {} does not exist or is not implemented.'.format(name))

        for feature in params['features']:
            add_by_name(self.features, feature)

        for target in params['targets']:
            add_by_name(self.targets, target)

        if set_h:
            params['h'] = self.h

        if set_shapes:
            params['features_shape'] = {name:feature.shape[1:] for name, feature in self.features.items()}
            params['targets_shape'] = {name:target.shape[1:] for name, target in self.targets.items()}

        if set_mean:
            params['features_mean'] = {name:np.mean(feature, axis=0) for name, feature in self.features.items()}
            params['targets_mean'] = {name:np.mean(target, axis=0) for name, target in self.targets.items()}


    @property
    def dataset(self):
        import tensorflow as tf
        return tf.data.Dataset.zip((tf.data.Dataset.zip({name:tf.data.Dataset.from_tensor_slices(feature) for name, feature in self.features.items()}), 
                                    tf.data.Dataset.zip({name:tf.data.Dataset.from_tensor_slices(target) for name, target in self.targets.items()})))
